In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# Carregar e preparar os dados de treino
train_data = pd.read_csv('imdb_data_train/imdb_data_train')
reviews = train_data['review'].tolist()
sentiments = train_data['sentiment'].tolist()

# Função de limpeza e tokenização
def clean_tokenize(document):
    document = re.sub('[^a-zA-Z]', ' ', document)
    tokens = word_tokenize(document.lower())
    cleaned_review = [word for word in tokens if word not in stopwords.words('english')]
    return cleaned_review

# Preparar os dados de treino
cleaned_data = [" ".join(clean_tokenize(review)) for review in reviews]
tfidfconverter = TfidfVectorizer(max_features=1500)
X_train = tfidfconverter.fit_transform(cleaned_data).toarray()
y_train = sentiments

# Inicializar e preparar os dados de teste
test_data = pd.read_csv('imdb_data_test/imdb_data_test')
test_reviews = test_data['review'].tolist()
test_sentiments = test_data['sentiment'].tolist()

# Preparar os dados de teste
cleaned_test_data = [" ".join(clean_tokenize(review)) for review in test_reviews]
X_test = tfidfconverter.transform(cleaned_test_data).toarray()
y_test = test_sentiments

# Inicializar o GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

grid_search = GridSearchCV(estimator=LogisticRegression(random_state=42), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Treinar o modelo com GridSearch
grid_search.fit(X_train, y_train)

# Melhores parâmetros e modelo
print("Melhores parâmetros: ", grid_search.best_params_)
best_classifier = grid_search.best_estimator_

# Fazer previsões e avaliar
y_pred = best_classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Salvar o modelo e o vetorizador
pickle.dump(best_classifier, open('Logistic_Regression_model.pkl', 'wb'))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\mcfar\pythonvenv\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mcfar\pythonvenv\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mcfar\pythonvenv\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\mcfar\pythonvenv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1169, in fit
    solver = _check_solver(self.solver, self.penalty, self.d

Melhores parâmetros:  {'C': 1, 'penalty': 'l2'}
[[10794  1706]
 [ 1523 10977]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87     12500
           1       0.87      0.88      0.87     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000
